Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [62]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [63]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
#train_dataset = train_dataset[0:500]
#train_labels = train_labels[0:500]
#train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [22]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [64]:
#logistic (gradient descent)
batch_size = 128
beta = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))#+tf.nn.l2_loss(weights)
  #l2_loss = tf.nn.l2_loss(weights)
  #loss = tf.reduce_mean(loss + beta*l2_loss)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

run logistic:

In [65]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.094543
Minibatch accuracy: 4.7%
Validation accuracy: 10.4%
Minibatch loss at step 500: 1.734971
Minibatch accuracy: 76.6%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.363147
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 1.388172
Minibatch accuracy: 74.2%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 1.017012
Minibatch accuracy: 82.8%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 1.239374
Minibatch accuracy: 71.1%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.932313
Minibatch accuracy: 79.7%
Validation accuracy: 78.7%
Test accuracy: 85.4%


neural network:

In [66]:
# init:
batch_size = 128
layer_size=1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer_size]))
  hidden_biases = tf.Variable(tf.zeros([layer_size]))
  hidden=tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights)+hidden_biases)
  output_weights = tf.Variable(
    tf.truncated_normal([layer_size, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  logits_relu = tf.matmul(hidden, output_weights) + output_biases
  
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits_relu, labels=tf_train_labels))
  l2_loss = tf.nn.l2_loss(hidden_weights)+tf.nn.l2_loss(output_weights)
  loss = tf.reduce_mean(loss + beta*l2_loss)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_relu)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_logits = tf.matmul(valid_hidden, output_weights) + output_biases
  valid_prediction= tf.nn.softmax(valid_logits)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_logits = tf.matmul(test_hidden, output_weights) + output_biases
  test_prediction = tf.nn.softmax(test_logits)

run it:

In [67]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3473.970459
Minibatch accuracy: 13.3%
Validation accuracy: 30.1%
Minibatch loss at step 500: 21.216183
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 0.971882
Minibatch accuracy: 81.2%
Validation accuracy: 84.1%
Minibatch loss at step 1500: 0.860821
Minibatch accuracy: 77.3%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 0.705797
Minibatch accuracy: 87.5%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 0.791795
Minibatch accuracy: 80.5%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.693367
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Minibatch loss at step 3500: 0.685011
Minibatch accuracy: 82.0%
Validation accuracy: 83.6%
Minibatch loss at step 4000: 0.901299
Minibatch accuracy: 79.7%
Validation accuracy: 82.2%
Minibatch loss at step 4500: 0.646005
Minibatch accuracy: 82.8%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 0.708889
Minibatch accuracy: 85.2%
Validation acc

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [69]:
num_steps = 5001
train_dataset2 = train_dataset[0:500]
train_labels2 = train_labels[0:500]

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset2[offset:(offset + batch_size), :]
    batch_labels = train_labels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3472.492920
Minibatch accuracy: 8.6%
Validation accuracy: 33.7%
Minibatch loss at step 500: 21.009111
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 0.462611
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 1500: 0.289550
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 2000: 0.272759
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.265937
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.267550
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3500: 0.272118
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 4000: 0.271137
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 4500: 0.264977
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 5000: 0.263853
Minibatch accuracy: 100.0%
Valid

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [79]:
# init:
batch_size = 128
layer_size=1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer_size]))
  hidden_biases = tf.Variable(tf.zeros([layer_size]))
  hidden=tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights)+hidden_biases)
  keep_prob = tf.placeholder("float")
  hidden_dropout=tf.nn.dropout(hidden,keep_prob)  
  output_weights = tf.Variable(
    tf.truncated_normal([layer_size, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  logits_relu = tf.matmul(hidden_dropout, output_weights) + output_biases
  
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits_relu, labels=tf_train_labels))
  #l2_loss = tf.nn.l2_loss(hidden_weights)+tf.nn.l2_loss(output_weights)
  #loss = tf.reduce_mean(loss + beta*l2_loss)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_relu)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_logits = tf.matmul(valid_hidden, output_weights) + output_biases
  valid_prediction= tf.nn.softmax(valid_logits)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_logits = tf.matmul(test_hidden, output_weights) + output_biases
  test_prediction = tf.nn.softmax(test_logits)

In [80]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 471.549927
Minibatch accuracy: 9.4%
Validation accuracy: 33.6%
Minibatch loss at step 500: 58.658638
Minibatch accuracy: 82.0%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 38.894203
Minibatch accuracy: 70.3%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 51.276627
Minibatch accuracy: 63.3%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 8.569421
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 9.750969
Minibatch accuracy: 66.4%
Validation accuracy: 80.1%
Minibatch loss at step 3000: 2.963427
Minibatch accuracy: 76.6%
Validation accuracy: 79.8%
Minibatch loss at step 3500: 6.571930
Minibatch accuracy: 75.8%
Validation accuracy: 79.6%
Minibatch loss at step 4000: 7.897675
Minibatch accuracy: 68.8%
Validation accuracy: 80.3%
Minibatch loss at step 4500: 2.893337
Minibatch accuracy: 68.8%
Validation accuracy: 80.9%
Minibatch loss at step 5000: 3.821286
Minibatch accuracy: 78.1%
Validation acc

In [81]:
#extreme overfitting
num_steps = 5001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset2[offset:(offset + batch_size), :]
    batch_labels = train_labels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 472.210022
Minibatch accuracy: 7.8%
Validation accuracy: 32.5%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Validat

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [83]:
# init:
batch_size = 128
layer_size=1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #weights = tf.Variable(
  global_step = tf.Variable(0)  # count the number of steps taken.  
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer_size]))
  hidden_biases = tf.Variable(tf.zeros([layer_size]))
  hidden=tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights)+hidden_biases)
  keep_prob = tf.placeholder("float")
  hidden_dropout=tf.nn.dropout(hidden,keep_prob)  
  output_weights = tf.Variable(
    tf.truncated_normal([layer_size, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))  
  # Training computation.
  logits_relu = tf.matmul(hidden_dropout, output_weights) + output_biases
  
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits_relu, labels=tf_train_labels))
  #l2_loss = tf.nn.l2_loss(hidden_weights)+tf.nn.l2_loss(output_weights)
  #loss = tf.reduce_mean(loss + beta*l2_loss)  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_relu)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_logits = tf.matmul(valid_hidden, output_weights) + output_biases
  valid_prediction= tf.nn.softmax(valid_logits)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_logits = tf.matmul(test_hidden, output_weights) + output_biases
  test_prediction = tf.nn.softmax(test_logits)

In [84]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 514.743652
Minibatch accuracy: 11.7%
Validation accuracy: 38.4%
Minibatch loss at step 500: 45.752609
Minibatch accuracy: 78.9%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 13.215878
Minibatch accuracy: 75.8%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 21.893135
Minibatch accuracy: 68.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 10.161316
Minibatch accuracy: 77.3%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 11.323671
Minibatch accuracy: 71.9%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 9.039368
Minibatch accuracy: 73.4%
Validation accuracy: 79.4%
Minibatch loss at step 3500: 6.939774
Minibatch accuracy: 76.6%
Validation accuracy: 80.8%
Minibatch loss at step 4000: 1.678041
Minibatch accuracy: 72.7%
Validation accuracy: 81.0%
Minibatch loss at step 4500: 1.745736
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 5000: 3.942953
Minibatch accuracy: 76.6%
Validation 